In [2]:
import os
import pickle
import pandas as pd
import numpy as np
import time
from utils import raw_data_path, feature_data_path,result_path,cache_pkl_path,dump_pickle,load_pickle
from sklearn.preprocessing import LabelEncoder

# 时间戳转字符串
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return dt

def extract_date(data):
    data['time'] = data.context_timestamp.apply(timestamp_datetime)
    data['day'] = data.time.apply(lambda x: int(x[8:10]))
    data['hour'] = data.time.apply(lambda x: int(x[11:13]))
    data['minute'] = data.time.apply(lambda x: int(x[14:15]))
    del data['time']
    return data

def gen_all_data():
    
    df_train = pd.read_csv(raw_data_path + "round1_ijcai_18_train_20180301.txt", sep=' ')
    df_test = pd.read_csv(raw_data_path + "round1_ijcai_18_test_a_20180301.txt", sep=' ')
    df_test_b = pd.read_csv(raw_data_path + "round1_ijcai_18_test_b_20180418.txt", sep=' ')
    
    df_train.drop_duplicates(inplace=True)
    df_test['is_trade'] = -1
    df_test_b['is_trade'] = -2

    all_data = pd.concat([df_train, df_test, df_test_b], ignore_index=True)
    all_data = extract_date(all_data)
    
    lb = LabelEncoder()
    all_data['category2_label'] = lb.fit_transform(all_data['item_category_list'].apply(lambda x:str(x).split(";")[1]))
    for i in range(8):
        all_data['item_property_list' + str(i)] = lb.fit_transform(all_data['item_property_list'].
                                                    apply(lambda x: str(str(x).split(';')[i]) if len(str(x).split(';')) > i else ''))
    dump_pickle(all_data, raw_data_path + 'all_data.pkl')

In [2]:
if __name__ == '__main__':

    gen_all_data()


In [3]:
all_data = load_pickle(raw_data_path + 'all_data.pkl')

In [17]:
 user_feature_click_day = all_data.groupby(['user_id', 'category2_label', 'day', 'item_brand_id']).size(
            ).reset_index().rename(columns={0: '_click_day'})

In [18]:
user_feature_click_day

,user_id,category2_label,day,item_brand_id,_click_day
0,7209196426168,11,25,8846021431831251875,1
1,24779788309075,10,19,7838285046767229711,3
2,36134987234568,9,22,3590481504050109449,1
3,59341486148291,11,22,4196025024202409809,1
4,174676690398750,11,25,5096509799201945977,1
5,179317972644611,11,22,3528918429809474042,1
6,179317972644611,11,22,4523623650186579300,1
7,179317972644611,11,22,5818352635473521241,1
8,179317972644611,11,22,7838285046767229711,2
9,179317972644611,11,22,7875739727730798464,1
